# Coursera Capstone - Battle of the Neighborhoods


## Introduction
This is the final project for the IBM Data Science Professional Certificate. I will be using Foursquare location data to explore neighborhoods in Milwaukee, WI.
## Background
A company notices the high growth rates of the number of young people in Milwaukee, WI, and wants to capitalize on it. They want to open a new restaurant in the city that will attract young people by providing a fun place to eat and hang out with their friends.
## Problem
The company is unfamiliar with the area and they need to choose the best location for the new restaurant so that it gets a lot of foot traffic and is in a convenient place for customers to get to. They have contracted our firm to find the best place for this new hang out.

 



## Data
For this analysis I will be using two data sets. The first data set is a list of all the neighborhoods in Milwaukee and their respective coordinates. This list also includes the district that the neighborhood is in. I used Wikipedia and Google to manually create this CSV file. Although the majority of neighborhoods had readily available coordinates, I had to base a few of them off locations that were in the neighborhood.

The second data source I will be using is venue data from the Foursquare API. I will use the neighborhood coordinates to query the API to get data about the venues near each location. 

![Neighborhoods](mke.png)
 



## Prepare Data
Import libararies

In [107]:
import requests
import folium

import pandas as pd
import numpy as np

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

In [108]:
#Read in neighborhood data

mke_data = pd.read_csv("MilwaukeeNeighborhoods.csv")
mke_data.rename(columns = {"NEIGHBORHOOD": "Neighborhood", "DISTRICT":"District"},inplace = True)
print(mke_data.shape)
mke_data.head()


(47, 4)


,District,Neighborhood,LAT,LONG
0,North Side,Arlington Heights,43.0861,-87.9284
1,North Side,Brewer's Hill,43.0563,-87.9114
2,North Side,Franklin Heights,43.0837,-87.9482
3,North Side,Granville,43.1775,-88.0440
4,North Side,Grover Heights,43.0917,-87.9200


## Map of Neighborhoods

In [109]:
#Milwaukee Coordinates
latitude = 43.0389
longitude = -87.9065

#Create map of Milwaukee
mke_map = folium.Map(location=[latitude, longitude], zoom_start=12)

#Add neighborhood markers to map
for lat, lng, district, neighborhood in zip(mke_data['LAT'], mke_data['LONG'], mke_data['District'], mke_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mke_map)

#Display map        
mke_map  

# Foursquare

In [110]:
import yaml
#Get foursquare credentials
with open("config.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

CLIENT_ID = cfg['api_creds']['client_id'] # your Foursquare ID
CLIENT_SECRET = cfg['api_creds']['client_secret']  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

## Get venue data

In [111]:
# get the 100 nearest venues for each neighborhood

LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius = 950):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [112]:
mke_venues = getNearbyVenues(names=mke_data['Neighborhood'],
                                   latitudes=mke_data['LAT'],
                                   longitudes=mke_data['LONG']
                                  )

Arlington Heights
Brewer's Hill
Franklin Heights
Granville
Grover Heights
Halyard Park
Harambee
Havenwoods
Hillside
Metcalfe Park
Midtown
Park West
Sherman Park
Williamsburg Heights
Bay View
Clarke Square
Holler Park
Jackson Park
Jones Island
Layton Park
Lincoln Village
Mitchell Street
Tippecanoe
Town of Lake
Walkers Point
The East Side
Lower East Side
East Village
Murray Hill
Riverwest
Avenues West
Cold Spring Park
Historic Concordia District
Enderis Park
Grantosa Heights
Kops Park
Martin Drive
Merrill Park
Mount Mary
Story Hill
Walnut Hill
Washington Heights
Washington Park
East Town
Menomonee River Valley
Historic Third Ward
Westown


In [113]:
# Check the venue data frame
print(mke_venues.shape)
mke_venues

(1846, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arlington Heights,43.0861,-87.9284,Mr. Perkin's Family Restaurant,43.090254,-87.937417,Southern / Soul Food Restaurant
1,Arlington Heights,43.0861,-87.9284,Family Dollar,43.089692,-87.938313,Discount Store
2,Arlington Heights,43.0861,-87.9284,Navy's Cafe,43.085612,-87.927986,Greek Restaurant
3,Arlington Heights,43.0861,-87.9284,Taco Truck,43.089388,-87.928537,Food Truck
4,Arlington Heights,43.0861,-87.9284,Redd Snapper,43.088413,-87.933052,Fish & Chips Shop
...,...,...,...,...,...,...,...
1841,Westown,43.0419,-87.9163,Cubanitas,43.039379,-87.906593,Cuban Restaurant
1842,Westown,43.0419,-87.9163,Blu Milwaukee,43.039361,-87.905690,Hotel Bar
1843,Westown,43.0419,-87.9163,Red White & Blue,43.044066,-87.914364,Bar
1844,Westown,43.0419,-87.9163,Wisconsin Athletic Club,43.038421,-87.905902,Gym


In [114]:
# number of venues in each neighborhood
mke_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arlington Heights,10,10,10,10,10,10
Avenues West,64,64,64,64,64,64
Bay View,89,89,89,89,89,89
Brewer's Hill,65,65,65,65,65,65
Clarke Square,50,50,50,50,50,50
Cold Spring Park,31,31,31,31,31,31
East Town,100,100,100,100,100,100
East Village,100,100,100,100,100,100
Enderis Park,17,17,17,17,17,17


In [115]:
print('There are {} uniques categories.'.format(len(mke_venues['Venue Category'].unique())))

There are 238 uniques categories.


# Analyzing neighborhoods

## One Hot encoding

In [116]:
# one hot encoding
mke_onehot = pd.get_dummies(mke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mke_onehot['Neighborhood'] = mke_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mke_onehot.columns[-1]] + list(mke_onehot.columns[:-1])
mke_onehot = mke_onehot[fixed_columns]

mke_onehot.shape

(1846, 238)

In [117]:
# frequency of each venue type by neighborhood
mke_grouped = mke_onehot.groupby('Neighborhood').mean().reset_index()
mke_grouped

,Neighborhood,Yoga Studio,ATM,African Restaurant,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Arcade,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Arlington Heights,0.000000,0.000000,0.00,0.000000,0.000000,0.100000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Avenues West,0.000000,0.015625,0.00,0.000000,0.000000,0.062500,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bay View,0.011236,0.000000,0.00,0.000000,0.000000,0.033708,0.00,0.000000,0.000000,...,0.011236,0.00,0.000000,0.011236,0.011236,0.000000,0.000000,0.000000,0.011236,0.000000
3,Brewer's Hill,0.000000,0.000000,0.00,0.000000,0.000000,0.015385,0.00,0.000000,0.015385,...,0.015385,0.00,0.000000,0.000000,0.000000,0.015385,0.015385,0.000000,0.000000,0.000000
4,Clarke Square,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,...,0.000000,0.00,0.020000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Cold Spring Park,0.000000,0.000000,0.00,0.000000,0.000000,0.032258,0.00,0.000000,0.000000,...,0.000000,0.00,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,East Town,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000
7,East Village,0.030000,0.000000,0.01,0.000000,0.000000,0.000000,0.00,0.010000,0.010000,...,0.010000,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Enderis Park,0.000000,0.058824,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Franklin Heights,0.000000,0.000000,0.00,0.000000,0.000000,0.117647,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [118]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Top venues in each neighborhood

In [131]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
mke_venues_sorted = pd.DataFrame(columns=columns)
mke_venues_sorted['Neighborhood'] = mke_grouped['Neighborhood']

for ind in np.arange(mke_grouped.shape[0]):
    mke_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mke_grouped.iloc[ind, :], num_top_venues)


mke_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arlington Heights,Fast Food Restaurant,American Restaurant,Southern / Soul Food Restaurant,Food Truck,Greek Restaurant,Park,Convenience Store,Flower Shop,Fish & Chips Shop,Discount Store
1,Avenues West,Sandwich Place,American Restaurant,Coffee Shop,Fast Food Restaurant,Bar,Brewery,Diner,History Museum,Pizza Place,Burger Joint
2,Bay View,Bar,Dive Bar,Park,Pet Store,Pizza Place,American Restaurant,Baseball Field,Coffee Shop,New American Restaurant,Mexican Restaurant
3,Brewer's Hill,Chinese Restaurant,Lounge,Coffee Shop,Performing Arts Venue,Playground,Park,Dance Studio,Bar,Gift Shop,Food Truck
4,Clarke Square,Mexican Restaurant,Discount Store,Laundromat,Cosmetics Shop,Bank,Gift Shop,Fast Food Restaurant,Mobile Phone Shop,Botanical Garden,Shop & Service


## Adding column to indicate younger population

In [120]:
#Find college campus or condos
df_college = mke_venues[mke_venues['Venue Category'].str.contains('Residential Building') | mke_venues['Venue Category'].str.contains('College')]
df_college.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
803,Lower East Side,43.05293,-87.89678,Landmark On The Lake,43.051906,-87.891368,Residential Building (Apartment / Condo)
898,East Village,43.05120,-87.89670,Landmark On The Lake,43.051906,-87.891368,Residential Building (Apartment / Condo)
1103,Avenues West,43.04100,-87.93720,Al McGuire Center,43.039889,-87.926806,College Basketball Court
1104,Avenues West,43.04100,-87.93720,Helfaer Recreation Center,43.037060,-87.933513,College Rec Center
1122,Avenues West,43.04100,-87.93720,McCormick Hall,43.039054,-87.932648,College Residence Hall


In [121]:
#Add column to indicate neighborhoods college campus or condos near by
cols = df_college['Neighborhood'].unique()
indice = mke_venues_sorted[mke_venues_sorted['Neighborhood'].isin(cols)].index.values

mke_venues_sorted['College'] = 'No'
mke_venues_sorted.loc[indice, 'College'] = 'Yes'

mke_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,College
0,Arlington Heights,Fast Food Restaurant,American Restaurant,Southern / Soul Food Restaurant,Food Truck,Greek Restaurant,Park,Convenience Store,Flower Shop,Fish & Chips Shop,Discount Store,No
1,Avenues West,Sandwich Place,American Restaurant,Coffee Shop,Fast Food Restaurant,Bar,Brewery,Diner,History Museum,Pizza Place,Burger Joint,Yes
2,Bay View,Bar,Dive Bar,Park,Pet Store,Pizza Place,American Restaurant,Baseball Field,Coffee Shop,New American Restaurant,Mexican Restaurant,No
3,Brewer's Hill,Chinese Restaurant,Lounge,Coffee Shop,Performing Arts Venue,Playground,Park,Dance Studio,Bar,Gift Shop,Food Truck,No
4,Clarke Square,Mexican Restaurant,Discount Store,Laundromat,Cosmetics Shop,Bank,Gift Shop,Fast Food Restaurant,Mobile Phone Shop,Botanical Garden,Shop & Service,No


## Clustering neighborhoods by venue types

In [122]:
# set number of clusters
kclusters = 5

mke_grouped_clustering = mke_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, n_init = 25, random_state=0).fit(mke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([1, 1, 1, 1, 2, 0, 1, 1, 2, 2], dtype=int32)

In [123]:
# add clustering labels
mke_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mke_data.set_index("Neighborhood", inplace = True)
# merge mke_grouped with mke_data to add latitude/longitude for each neighborhood
mke_merged = mke_data.join(mke_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [124]:
mke_merged.head()

,District,LAT,LONG,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,College
Neighborhood,,,,,,,,,,,,,,,
Arlington Heights,North Side,43.0861,-87.9284,1,Fast Food Restaurant,American Restaurant,Southern / Soul Food Restaurant,Food Truck,Greek Restaurant,Park,Convenience Store,Flower Shop,Fish & Chips Shop,Discount Store,No
Brewer's Hill,North Side,43.0563,-87.9114,1,Chinese Restaurant,Lounge,Coffee Shop,Performing Arts Venue,Playground,Park,Dance Studio,Bar,Gift Shop,Food Truck,No
Franklin Heights,North Side,43.0837,-87.9482,2,Fast Food Restaurant,Liquor Store,Pizza Place,American Restaurant,Clothing Store,Fried Chicken Joint,Diner,Grocery Store,Home Service,Convenience Store,No
Granville,North Side,43.1775,-88.0440,4,Pub,Mexican Restaurant,Liquor Store,Bar,Asian Restaurant,Storage Facility,Sandwich Place,Construction & Landscaping,Miscellaneous Shop,Ethiopian Restaurant,No
Grover Heights,North Side,43.0917,-87.9200,2,Gym / Fitness Center,Bank,Fast Food Restaurant,Park,Brewery,Flower Shop,Shipping Store,Music Store,Food Truck,Mobile Phone Shop,No


## Plot neighborhoods with cluster labels

In [125]:
# create map
cluster_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# get number of clusters
kclusters = mke_merged["Cluster Labels"].max()+1

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mke_merged['LAT'], mke_merged['LONG'], mke_merged.index.values ,mke_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ': Cluster ' + str(cluster+1), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster_map)
       
cluster_map

# Specific cluster data

## Cluster 1

In [ ]:
for ind in np.arange(mke_grouped.shape[0]):
    mke_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mke_grouped.iloc[ind, :], num_top_venues)

In [126]:
mke_merged.loc[mke_merged['Cluster Labels'] == 0, mke_merged.columns[list(range(3, mke_merged.shape[1]))]]


,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,College
Neighborhood,,,,,,,,,,,,
Midtown,0,Park,Convenience Store,Pharmacy,Business Service,Sports Bar,Liquor Store,Fried Chicken Joint,Bakery,Garden,Salon / Barbershop,No
Jackson Park,0,Bar,Park,Pizza Place,Playground,Gun Shop,Pet Store,Music Store,Soccer Field,Supermarket,Discount Store,No
Tippecanoe,0,Bar,Dive Bar,Liquor Store,Basketball Court,Home Service,Pizza Place,Park,Convenience Store,Clothing Store,Discount Store,No
Riverwest,0,Bar,Grocery Store,Trail,Rock Club,Brewery,Convenience Store,Coffee Shop,Café,Discount Store,Art Gallery,No
Cold Spring Park,0,Bed & Breakfast,Liquor Store,Discount Store,Bar,Pool Hall,Chinese Restaurant,Steakhouse,Convenience Store,Sports Bar,Fast Food Restaurant,No
Historic Concordia District,0,Bed & Breakfast,Bar,Beer Garden,Discount Store,Bus Stop,Brewery,Liquor Store,Southern / Soul Food Restaurant,Fast Food Restaurant,Chinese Restaurant,No
Martin Drive,0,Brewery,Park,Beer Garden,Cosmetics Shop,Bar,Automotive Shop,Seafood Restaurant,Snack Place,Liquor Store,Food Court,No
Merrill Park,0,Bar,Bed & Breakfast,Liquor Store,Beer Garden,Brewery,Bus Stop,Home Service,Chinese Restaurant,Southern / Soul Food Restaurant,Scenic Lookout,No
Story Hill,0,Bar,Sports Bar,Brewery,Park,Gift Shop,Outdoor Sculpture,Baseball Stadium,Electronics Store,Bus Stop,Pub,No


## Cluster 2

In [127]:
mke_merged.loc[mke_merged['Cluster Labels'] == 1, mke_merged.columns[list(range(4, mke_merged.shape[1]))]]

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,College
Neighborhood,,,,,,,,,,,
Arlington Heights,Fast Food Restaurant,American Restaurant,Southern / Soul Food Restaurant,Food Truck,Greek Restaurant,Park,Convenience Store,Flower Shop,Fish & Chips Shop,Discount Store,No
Brewer's Hill,Chinese Restaurant,Lounge,Coffee Shop,Performing Arts Venue,Playground,Park,Dance Studio,Bar,Gift Shop,Food Truck,No
Halyard Park,Fast Food Restaurant,Performing Arts Venue,Shoe Store,Lounge,Fried Chicken Joint,Food Truck,Café,Restaurant,Business Service,Bar,No
Hillside,Brewery,Fast Food Restaurant,Performing Arts Venue,Bar,Coffee Shop,Food Truck,Restaurant,Gas Station,Furniture / Home Store,Deli / Bodega,No
Sherman Park,Restaurant,Bar,Cosmetics Shop,Fast Food Restaurant,Park,Furniture / Home Store,Fried Chicken Joint,Food Court,Grocery Store,Sports Bar,No
Bay View,Bar,Dive Bar,Park,Pet Store,Pizza Place,American Restaurant,Baseball Field,Coffee Shop,New American Restaurant,Mexican Restaurant,No
Holler Park,Airport Terminal,Airport Service,American Restaurant,Hotel,Home Service,Clothing Store,Shipping Store,Food & Drink Shop,Burger Joint,Laser Tag,No
Town of Lake,Bar,Fast Food Restaurant,Pizza Place,Sandwich Place,Flower Shop,Dive Bar,Gas Station,Basketball Court,Discount Store,Sports Bar,No
The East Side,Bar,Coffee Shop,Pizza Place,Ice Cream Shop,Movie Theater,Café,Salon / Barbershop,Japanese Restaurant,Jazz Club,Beer Bar,No


## Cluster 3

In [128]:
mke_merged.loc[mke_merged['Cluster Labels'] == 2, mke_merged.columns[list(range(4, mke_merged.shape[1]))]]

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,College
Neighborhood,,,,,,,,,,,
Franklin Heights,Fast Food Restaurant,Liquor Store,Pizza Place,American Restaurant,Clothing Store,Fried Chicken Joint,Diner,Grocery Store,Home Service,Convenience Store,No
Grover Heights,Gym / Fitness Center,Bank,Fast Food Restaurant,Park,Brewery,Flower Shop,Shipping Store,Music Store,Food Truck,Mobile Phone Shop,No
Metcalfe Park,Sandwich Place,Video Store,Women's Store,Grocery Store,History Museum,Cosmetics Shop,Discount Store,Burger Joint,Pharmacy,American Restaurant,No
Park West,Grocery Store,History Museum,BBQ Joint,Convenience Store,Coffee Shop,Deli / Bodega,Sandwich Place,Burger Joint,Farmers Market,Food Truck,No
Williamsburg Heights,Gym / Fitness Center,Grocery Store,Liquor Store,Fried Chicken Joint,Bank,Home Service,Bakery,Park,Paper / Office Supplies Store,Clothing Store,No
Clarke Square,Mexican Restaurant,Discount Store,Laundromat,Cosmetics Shop,Bank,Gift Shop,Fast Food Restaurant,Mobile Phone Shop,Botanical Garden,Shop & Service,No
Mitchell Street,Fast Food Restaurant,Pizza Place,Latin American Restaurant,Cocktail Bar,Gas Station,Discount Store,Farmers Market,Bank,Mexican Restaurant,Bowling Alley,No
Enderis Park,Park,Playground,Fast Food Restaurant,Greek Restaurant,Bank,Liquor Store,Credit Union,Sports Bar,Convenience Store,Pizza Place,No


## Cluster 4

In [129]:
mke_merged.loc[mke_merged['Cluster Labels'] == 3, mke_merged.columns[list(range(4, mke_merged.shape[1]))]]

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,College
Neighborhood,,,,,,,,,,,
Jones Island,Boat or Ferry,Park,Other Great Outdoors,Seafood Restaurant,Harbor / Marina,Distillery,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,No


## Cluster 5

In [130]:
mke_merged.loc[mke_merged['Cluster Labels'] == 4, mke_merged.columns[list(range(4, mke_merged.shape[1]))]]

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,College
Neighborhood,,,,,,,,,,,
Granville,Pub,Mexican Restaurant,Liquor Store,Bar,Asian Restaurant,Storage Facility,Sandwich Place,Construction & Landscaping,Miscellaneous Shop,Ethiopian Restaurant,No
Harambee,Bar,Grocery Store,Pharmacy,Trail,Lounge,Sandwich Place,Bank,Rock Club,Café,Intersection,No
Havenwoods,Baseball Field,Trail,Intersection,Hobby Shop,Garden,Bowling Alley,Convenience Store,Park,Furniture / Home Store,Thrift / Vintage Store,No
Layton Park,Mexican Restaurant,Bar,Bakery,Pharmacy,Food & Drink Shop,Thrift / Vintage Store,Intersection,River,Soccer Field,Food,No
Lincoln Village,Mexican Restaurant,Park,Bar,Pizza Place,Grocery Store,Locksmith,Latin American Restaurant,Deli / Bodega,Bowling Alley,Tennis Court,No
Walkers Point,Mexican Restaurant,Bar,Brewery,Grocery Store,Cocktail Bar,Nightclub,Lounge,Restaurant,Burger Joint,Art Gallery,No
Grantosa Heights,Cosmetics Shop,Food & Drink Shop,Grocery Store,Park,Bar,Gas Station,Chinese Restaurant,Sandwich Place,Pizza Place,Convenience Store,No
Kops Park,Park,Pizza Place,Chinese Restaurant,Mexican Restaurant,Sandwich Place,Bar,Café,Soccer Field,Stationery Store,Discount Store,No
Mount Mary,Gas Station,Mexican Restaurant,Furniture / Home Store,Pharmacy,Park,Coffee Shop,Sandwich Place,Café,Baseball Field,Trail,No


# Conclusion

We need to look for the best place to locate a new restaurant/hangout in Milwaukee, WI.

Looking at the specific cluster data, we can quickly rule out cluster 4 as that is a ferry port with not much else near.

Clusters 3 and 5 have much more residential and daytime venues, such as parks, gyms, shops, and grocery stores.

Depending on the type of restaurant that is being opened, I would recommend either cluster 1 or 2. Cluster 1 is almost all bars, which would attract a lot of people to those areas at night, while cluster 2 is mostly restaurant and coffe shops, which would see more traffic during the day. In addition, cluster 2 has the University of Milwaukee campus, and college students love to go out and explore new places.